In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
f_exp = pd.read_csv('experience_pdl.csv',dtype=str)

In [3]:
#Translate the string to datetime
for index, row in f_exp.iterrows():
    row['start_date'] = str(row['start_date'])
    if len(row['start_date'])==4:
        row['start_date'] = row['start_date'] + '-06'
    elif len(row['start_date']) > 7:
        row['start_date'] = row['start_date'][7]

In [4]:
#transform all datetime to YYYY-mm format
error1 = ['']
for index, row in f_exp.iterrows():
    try:
        row['start_date'] = datetime.strptime(row['start_date'], '%Y-%m')
    except:
        error1.append(row.profile_id)

In [5]:
for index, row in f_exp.iterrows():
    row['end_date'] = str(row['end_date'])
    if len(row['end_date'])==4:
        row['end_date'] = row['end_date'] + '-06'
    elif len(row['end_date']) > 7:
        row['end_date'] = row['end_date'][7]

In [6]:
error2 = ['']
for index, row in f_exp.iterrows():
    try:
        row['end_date'] = datetime.strptime(row['end_date'], '%Y-%m')
    except:
        error2.append(row.profile_id)

In [7]:
error_ed = pd.DataFrame({'id':error2})

In [8]:
#some null value may became string while being read, fix this
f_exp['start_date'] = f_exp['start_date'].replace('nan',np.NaN)
f_exp['start_date'] = f_exp['start_date'].replace('-',np.NaN)
f_exp['end_date'] = f_exp['end_date'].replace('nan',np.NaN)
f_exp['end_date'] = f_exp['end_date'].replace('-',np.NaN)

In [9]:
#delete all internship experience
f_exp['function'] = f_exp['function'].fillna('Unknown')
f_exp = f_exp[~f_exp['function'].str.contains("intern","internship")]

In [10]:
#Create a copy for later use
exp1 = f_exp

In [11]:
#how many id have exp missing start_date
len(set(exp1.loc[exp1[['start_date']].isna().any(axis=1),'profile_id']))

489652

In [12]:
#how many id have exp missing both start_date & end_date
len(set(exp1.loc[exp1[['start_date','end_date']].isnull().all(1), 'profile_id']))

488804

In [13]:
exp2 = exp1[~exp1[['start_date','end_date']].isnull().all(1)]
a = exp1.profile_id.nunique()
b = exp2.profile_id.nunique()

#How many id have only one exp which has no date at all
print(a, '-', b, '=', a-b)

1005239 - 679853 = 325386


In [14]:
id_misd = np.intersect1d(exp2.profile_id.unique(),exp1.loc[exp1[['start_date','end_date']].isnull().all(1), 'profile_id'])
id_misd

array(['--1VK_NnS8SJwP3ZFk3i', '--3MYrYWTxexhOr5-D7I',
       '--4x07mPRXGY12R_RjTr', ..., 'zzwb7vgxSoyPaINAN8bZ',
       'zzxJxYnoSZadOGzfzpse', 'zzyjDuP9TxW0YPRsnilS'], dtype=object)

In [15]:
len(id_misd)

163418

In [16]:
'''
Solution to missing start_date problem:
1. Missing only start date
2. Missing both sd and ed and being the only exp
3. Missing both sd and ed but not the only exp
'''

s1 = set(exp1.loc[exp1[['start_date']].isna().any(axis=1),'profile_id'])
s2 = set(exp1.loc[exp1[['start_date','end_date']].isnull().all(1), 'profile_id'])

In [17]:
e1 = set.difference(s1,s2)
len(e1)

848

In [18]:
#problem 1, small amount but too much trouble. So just drop them all

exp1 = exp1[~exp1['profile_id'].isin(e1)]

In [19]:
#After that, problem 2 & 3,just drop relevant exp rows and keep ids(if not the only exp)
exp11 = exp1[exp1['start_date'].notna()]

In [20]:
exp11['profile_id'].nunique()

678996

In [21]:
exp11['end_date'].isna().sum()

862111

In [22]:
#if last exp has not ended, drop it. if not and still missing end date, fill in with start date of next exp

exp11 = exp11.reset_index(drop=True)
exp11 = exp11.sort_values(by=['profile_id','start_date'])

In [23]:
#get index and id of missing end_date exp
index_ed = exp11.groupby('profile_id').tail(1).loc[lambda x : x.end_date.isna()].index
id_ed = exp11.groupby('profile_id').tail(1).loc[lambda x : x.end_date.isna()].profile_id

In [24]:
#save the dropped record's company and profile_id for double-check later
df_check = exp11.loc[index_ed]

In [25]:
exp11.head(10)

,profile_id,company,size,found_year,industry,hq_locality,hq_region,hq_country,website,location,...,subregion,country,continent,start_date,end_date,title,level,function,type,exp_desc
1618599,---7ZN5QQ6Ots7_KAGrz,"roschier, attorneys ltd.",501-1000,1936.0,legal services,helsinki,uusimaa,finland,roschier.com,NaN,...,NaN,NaN,NaN,2012-01-01 00:00:00,2012-06-01 00:00:00,associate trainee,NaN,associate\ntrainee,NaN,NaN
1618598,---7ZN5QQ6Ots7_KAGrz,kpmg,10001+,NaN,financial services,new york,new york,united states,beaconpartners.com,NaN,...,NaN,NaN,NaN,2012-07-01 00:00:00,2012-09-01 00:00:00,legal trainee,NaN,legal trainee,NaN,NaN
1618597,---7ZN5QQ6Ots7_KAGrz,hannes snellman,501-1000,1909.0,legal services,helsinki,uusimaa,finland,hannessnellman.com,NaN,...,NaN,NaN,NaN,2013-01-01 00:00:00,2013-04-01 00:00:00,associate trainee,NaN,associate\ntrainee,NaN,NaN
1618596,---7ZN5QQ6Ots7_KAGrz,kalliolaw asianajotoimisto oy - attorneys at law,11-50,2000.0,legal services,helsinki,uusimaa,finland,kalliolaw.fi,NaN,...,NaN,NaN,NaN,2014-01-01 00:00:00,NaN,associate,NaN,associate,NaN,NaN
1820435,---GzVTjTQGc0JuhWqZn,central chambers law corporation,11-50,NaN,law practice,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2008-05-01 00:00:00,NaN,legal secretary,NaN,legal secretary,NaN,NaN
330624,---NvmpCRJ2v09r1vf_v,experd consulting,201-500,1989.0,human resources,jakarta,jakarta raya,indonesia,experd.com,yogyakarta,...,NaN,indonesia,asia,2007-06-01 00:00:00,2007-06-01 00:00:00,associate,NaN,associate,NaN,NaN
330619,---NvmpCRJ2v09r1vf_v,bambini montessori school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2007-07-01 00:00:00,2008-06-01 00:00:00,assistant teacher of kindergarten a class,NaN,kindergarten\nassistant\nteacher,NaN,"assisted the main teacher /educator, prepared ..."
330618,---NvmpCRJ2v09r1vf_v,university of melbourne,10001+,NaN,higher education,parkville,new south wales,australia,unimelb.edu.au,melbourne,...,melbourne,australia,oceania,2009-01-01 00:00:00,2009-12-01 00:00:00,peer-assisted study and skill integration lead...,mgr,study\nintegration leader\npart,NaN,"facilitated, led, and organized group discussi..."
330617,---NvmpCRJ2v09r1vf_v,universal restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,melbourne,...,melbourne,australia,oceania,2010-06-01 00:00:00,2011-02-01 00:00:00,bartender (part-time),NaN,bartender\npart,NaN,NaN
330623,---NvmpCRJ2v09r1vf_v,mercury group of companies,11-50,1997.0,information technology and services,melbourne,victoria,australia,mercury.com.au,melbourne,...,melbourne,australia,oceania,2011-02-01 00:00:00,2012-02-01 00:00:00,office administrator/ recruitment researcher,NaN,office administrator\nrecruitment researcher,NaN,NaN


In [26]:
exp11['fillna'] = exp11['start_date']

In [27]:
exp11.tail(5)

,profile_id,company,size,found_year,industry,hq_locality,hq_region,hq_country,website,location,...,country,continent,start_date,end_date,title,level,function,type,exp_desc,fillna
1615409,zzzbmBygSSCMNW70J8rM,pt. ishida indonesia,1-10,2012.0,machinery,jakarta,jakarta raya,indonesia,ishida.com,NaN,...,NaN,NaN,2015-04-01 00:00:00,2017-03-01 00:00:00,chief sales & marketing/supervisor,mgr,sales\nchief\nmarketing\nsupervisor,NaN,-managing sales staff\n-coordinating sales act...,2015-04-01 00:00:00
1615406,zzzbmBygSSCMNW70J8rM,pt. ishida indonesia,1-10,2012.0,machinery,jakarta,jakarta raya,indonesia,ishida.com,NaN,...,NaN,NaN,2017-04-01 00:00:00,2017-07-01 00:00:00,assistant sales & marketing manager,NaN,assistant sales\nmarketing manager,NaN,NaN,2017-04-01 00:00:00
1615408,zzzbmBygSSCMNW70J8rM,go-jek,1001-5000,2010.0,information technology and services,jakarta,jakarta raya,indonesia,go-jek.com,NaN,...,NaN,NaN,2017-07-01 00:00:00,2018-03-01 00:00:00,senior account manager,mgr,senior account manager,NaN,go-food smb sales division,2017-07-01 00:00:00
1615407,zzzbmBygSSCMNW70J8rM,go-jek,1001-5000,2010.0,information technology and services,jakarta,jakarta raya,indonesia,go-jek.com,NaN,...,NaN,NaN,2018-04-01 00:00:00,2018-08-01 00:00:00,merchant acquisition manager,mgr,acquisition manager\nmerchant,NaN,go-food smb sales division,2018-04-01 00:00:00
1615405,zzzbmBygSSCMNW70J8rM,go-jek,1001-5000,2010.0,information technology and services,jakarta,jakarta raya,indonesia,go-jek.com,NaN,...,NaN,NaN,2018-10-01 00:00:00,NaN,business development manager - go-food festival,mgr,business development manager\nfood,NaN,NaN,2018-10-01 00:00:00


In [28]:
exp11['fillna'] = exp11['fillna'].shift(-1)

In [29]:
exp11['end_date'] = exp11['end_date'].fillna(exp11['fillna'])

In [30]:
#Drop the rows in index_ed
exp11 = exp11[~exp11.index.isin(index_ed)]

In [31]:
exp11['end_date'].isna().sum()

0

In [32]:
#drop fillna column
exp11 = exp11.drop('fillna',axis=1)

In [33]:
#Problem: A lot of s/e dates filled with same yyyy, will causes 0 tenure, choose to drop
#Problem: Exist same company multiple experience
exp11 = exp11[exp11['start_date'] != exp11['end_date']]

In [34]:
#there are missing values in the 'company' column, drop all the id have this error
error_c = set(exp11.loc[exp11[['company']].isna().any(axis=1),'profile_id'])
exp11 = exp11[~exp11['profile_id'].isin(error_c)]

In [35]:
exp11['company'].isnull().sum()

0

In [36]:
exp11['work_id'] = exp11['profile_id'] + exp11['company']

In [37]:
#build a new table without duplicated experience
idx1 = exp11.groupby(['profile_id','company'])['end_date'].transform(max) == exp11['end_date']
df1 = exp11[idx1]

In [38]:
idx2 = exp11.groupby(['profile_id','company'])['start_date'].transform(min) == exp11['start_date']
df2 = exp11[idx2]

In [39]:
df11 = df1.drop_duplicates(subset=['company','profile_id'])

In [40]:
df22 = df2.drop_duplicates(subset=['company','profile_id'])

In [41]:
df11 = df11.drop(['start_date'],axis=1)

In [42]:
df11.shape

(1713735, 21)

In [43]:
df22['work_id'].nunique()

1713735

In [44]:
df22 = df22[['work_id','start_date']]

In [45]:
df11 = df11.merge(df22, how = 'left', on = ['work_id'])

In [46]:
df11 = df11.sort_values(by=['profile_id','start_date'])

In [47]:
df11['tenure'] = df11['end_date'] - df11['start_date']
df11['tenure'] = df11['tenure'].dt.days

In [48]:
df11.shape

(1713735, 23)

In [49]:
#Double check previous ids to see whether dropping last exp with null end_date cause target tenure < 1y
df_check = df_check[['profile_id','company']]
df_check['work_id'] = df_check['profile_id'] + df_check['company']

In [50]:
df_check['work_id'].nunique()

546903

In [51]:
df11['work_id'].nunique()

1713735

In [52]:
check = set(df_check['work_id'])

In [53]:
df3 = df11[(df11['work_id'].isin(check)) & (df11['tenure'] < 365)]

In [54]:
# number of rows that should be dropped
df3.shape

(15830, 23)

In [55]:
data = df11[~df11.index.isin(df3.index)]

In [56]:
data.shape

(1697905, 23)

In [57]:
#delete id that has exp dates before 1950
t1 = datetime.strptime('1950-01-01','%Y-%m-%d')

df_t1 = data[(data['start_date']< t1 ) | (data['end_date'] < t1)]

In [58]:
data = data[~data['profile_id'].isin(df_t1['profile_id'])]

In [59]:
data.shape

(1697689, 23)

In [60]:
#find the target exp
data = data.sort_values(by = ['start_date'])
t_index = data.groupby('profile_id').tail(1).index
df_target = data[data.index.isin(t_index)]

In [104]:
df_all = df_target[['profile_id','company','size','industry','hq_country','website','level','tenure']]

In [105]:
#delete the data with negative tenure
df_all = df_all[df_all['tenure'] > 0]

In [106]:
df_all.shape

(529132, 8)

In [107]:
#drop the target exp to calculate the total and average tenure
data = data[~data.index.isin(t_index)]

In [108]:
df_sum = data.groupby('profile_id')['tenure'].sum()
df_avg = data.groupby('profile_id')['tenure'].mean()

In [109]:
df_sum

profile_id
---7ZN5QQ6Ots7_KAGrz     214
---NvmpCRJ2v09r1vf_v    2253
--1GeOqPT9uBYjj_r_wQ     792
--1sqVPNQ52JLooM2q3F    2099
--24Bl1SQsGQ7InMvsgD    3014
                        ... 
zzwrn9BpQqWTHWkfUSNM    1247
zzxByvUCTFqoXACujJqC    5264
zzxJxYnoSZadOGzfzpse    4078
zzy9TldLTEmwJcCy1CGt     855
zzzbmBygSSCMNW70J8rM    1065
Name: tenure, Length: 392578, dtype: int64

In [110]:
df_all = df_all.merge(df_sum, on=['profile_id'], how = 'left')
df_all = df_all.merge(df_avg, on=['profile_id'], how = 'left')

In [111]:
df_all = df_all.rename(columns={"tenure_x": "tenure", "tenure_y": "tot_tenure","tenure":"avg_tenure"})

In [112]:
df_all['target'] = np.where((df_all['tenure']>365), 'loyal', 'disloyal')

In [113]:
#Get profile info
df_pro = pd.read_csv('profile_pdl.csv')

df_pro = df_pro[['profile_id','salary','country','continent']]

In [114]:
df_all = df_all.merge(df_pro, on=['profile_id'], how = 'left')

#Get industry info
df_ind = pd.read_csv('industry_pdl.csv')
df_ind = df_ind.rename(columns={'id':'profile_id'})
df_all = df_all.merge(df_ind, on=['profile_id'], how = 'left')

In [115]:
#Get interest info
int = pd.read_csv('interest_pdl.csv')
int = int.rename(columns={'id':'profile_id'})
int = int.drop('Unnamed: 0', axis=1)
df_int = pd.DataFrame(int.groupby('profile_id').count())
df_int = df_int.reset_index(drop = False)

In [116]:
df_all = df_all.merge(df_int, on=['profile_id'], how = 'left')
df_all = df_all.rename(columns={"interest": "int_count"})

In [117]:
#Get skill info
skill = pd.read_csv('skill_pdl.csv')
skill = skill.rename(columns={'id':'profile_id'})

df_ski = pd.DataFrame(skill.groupby('profile_id').count())
df_ski = df_ski.reset_index(drop = False)

In [118]:
skill['skill'].nunique()

223160

In [119]:
df_all = df_all.merge(df_ski, on=['profile_id'], how = 'left')
df_all = df_all.rename(columns={"skill": "skill_count"})

In [120]:
df_all

,profile_id,company,size,industry,hq_country,website,level,tenure,tot_tenure,avg_tenure,target,salary,country,continent,int_count,skill_count
0,TX2lkDILT7ipMZ_6jt6X,panti asuhan kerahiman hawai papua,NaN,NaN,NaN,NaN,NaN,22646,NaN,NaN,loyal,NaN,indonesia,asia,NaN,NaN
1,-drFXizgQaSS6QApPYZQ,autoprofit marketing pte,NaN,NaN,NaN,NaN,NaN,21823,NaN,NaN,loyal,"55,000-70,000",singapore,asia,NaN,8.0
2,DY0Ed5iVQcy15UPDS1i7,cherrybelle,NaN,NaN,NaN,NaN,NaN,2192,NaN,NaN,loyal,NaN,indonesia,asia,NaN,NaN
3,HelFHtkjT0C-ab89XR7m,"second to none, inc.",1001-5000,market research,united states,second-to-none.com,NaN,6210,NaN,NaN,loyal,NaN,indonesia,asia,NaN,6.0
4,17tAIFLSQCWi7SNRapja,microsoft,10001+,computer software,united states,microsoft.com,c,22281,NaN,NaN,loyal,NaN,singapore,asia,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529127,Ki682VYoQOy8-R0YQPJh,millennial board,51-200,think tanks,finland,millennialboard.fi,NaN,61,3016.0,430.857143,disloyal,"85,000-100,000",finland,europe,10.0,27.0
529128,CgNuZexDR0C8dGR8RinZ,ylöjärven ilves ry,NaN,NaN,NaN,NaN,NaN,30,2496.0,832.000000,disloyal,"85,000-100,000",finland,europe,NaN,27.0
529129,psd-ZhaITQGnkRuJsXn3,university of indonesia,5001-10000,higher education,indonesia,ui.ac.id,NaN,30,1312.0,218.666667,disloyal,"45,000-55,000",indonesia,asia,14.0,30.0
529130,1QMowLw0QJKbpSRhZN25,slush,201-500,information technology and services,finland,slush.org,NaN,30,1767.0,294.500000,disloyal,"70,000-85,000",finland,europe,NaN,19.0


In [121]:
#Get education info
edu = pd.read_csv('education_pdl.csv')

In [122]:
df_edu = edu[['profile_id','qual_type','degree']]

In [123]:
df_edu = df_edu[df_edu['qual_type'].notna()]

In [124]:
df_edu.degree.nunique()

13900

In [125]:
df_edu.loc[(df_edu['qual_type'].str.contains("associate")), 'qual_type'] = 'associate'

df_edu.loc[(df_edu['qual_type'].str.contains("bachelor")), 'qual_type'] = 'bachelor'

df_edu.loc[(df_edu['qual_type'].str.contains("master")), 'qual_type'] = 'master'

df_edu.loc[(df_edu['qual_type'].str.contains("doctor")), 'qual_type'] = 'doctor'

In [126]:
df_edu = df_edu.replace('magisters\nmagister juris', 'master')

In [127]:
#Find highest degree of each individual
df_edu['seq'] = df_edu['qual_type']
df_edu['seq'] = df_edu['seq'].replace({'associate':1, 'bachelor':2, 'master':3, 'doctor':4})

In [128]:
df_edu = df_edu.sort_values(by = ['seq'])

df_edu = df_edu.drop_duplicates(subset=['profile_id'], keep = 'last')

In [129]:
df_all = df_all.merge(df_edu, on=['profile_id'], how = 'left')

In [130]:
df_all = df_all.drop('seq',axis=1)

In [131]:
#CLean 'size'
df_all['size'] = df_all['size'].fillna('Unknown')
df_all['size'].unique()

array(['Unknown', '1001-5000', '10001+', '5001-10000', '1-10', '501-1000',
       '201-500', '51-200', '11-50'], dtype=object)

In [135]:
df_all.head(5)

,profile_id,company,size,industry,hq_country,website,level,tenure,tot_tenure,avg_tenure,target,salary,country,continent,int_count,skill_count,qual_type,degree
0,TX2lkDILT7ipMZ_6jt6X,panti asuhan kerahiman hawai papua,Unknown,NaN,NaN,NaN,NaN,22646,NaN,NaN,loyal,NaN,indonesia,asia,NaN,NaN,NaN,NaN
1,-drFXizgQaSS6QApPYZQ,autoprofit marketing pte,Unknown,NaN,NaN,NaN,NaN,21823,NaN,NaN,loyal,"55,000-70,000",singapore,asia,NaN,8.0,NaN,NaN
2,DY0Ed5iVQcy15UPDS1i7,cherrybelle,Unknown,NaN,NaN,NaN,NaN,2192,NaN,NaN,loyal,NaN,indonesia,asia,NaN,NaN,NaN,NaN
3,HelFHtkjT0C-ab89XR7m,"second to none, inc.",1001-5000,market research,united states,second-to-none.com,NaN,6210,NaN,NaN,loyal,NaN,indonesia,asia,NaN,6.0,NaN,NaN
4,17tAIFLSQCWi7SNRapja,microsoft,10001+,computer software,united states,microsoft.com,c,22281,NaN,NaN,loyal,NaN,singapore,asia,NaN,3.0,NaN,NaN


In [ ]:
df_all['hq_country'].unique()

In [138]:
df_all['have_web'] = np.where((df_all['website'].isna()), 'N', 'Y')

In [140]:
df_all = df_all.drop('website',axis=1)

In [150]:
df_all['title'] = df_all['level']

In [152]:
df_all['title'] = df_all['title'].replace({ 'c\ndir':'c', 'vp\ndir':'vp',
       'mgr\nvp':'mgr', 'pres\ndir':'pres', 'mgr\ndir':'mgr', 'mgr\nc':'mgr',
       'pres\nowner':'pres', 'owner\ndir':'owner', 'c\nowner':'c', 'pres\nc':'pres', 'vp\nc':'vp',
       'mgr\nc\ndir':'mgr', 'owner\nc':'owner', 'mgr\nowner':'mgr', 'mgr\npres':'mgr', 'mgr\nvp\nc':'mgr',
       'pres\nc\ndir':'pres', 'dir\nc\nowner':'dir', 'mgr\nowner\ndir':'mgr', 'vp\npres':'vp',
       'vp\nowner':'vp', 'pres\nc\nowner':'pres', 'mgr\npres\ndir':'mgr', 'owner\nc\ndir':'owner',
       'vp\nowner\ndir':'vp', 'mgr\nvp\ndir':'mgr', 'mgr\nowner\nc':'mgr', 'mgr\nc\nowner':'mgr',
       'pres\nowner\ndir':'pres', 'dir\nc':'dir'})

In [155]:
df_all['title'] = df_all['title'].fillna('none')

In [157]:
df_all['tot_tenure'] = df_all['tot_tenure'].fillna(0)

In [158]:
df_all['avg_tenure'] = df_all['avg_tenure'].fillna(0)

In [160]:
df_all['salary'] = df_all['salary'].fillna('Unknown')

In [163]:
df_all['int_count'] = df_all['int_count'].fillna(0)

In [164]:
df_all['skill_count'] = df_all['skill_count'].fillna(0)

In [166]:
df_all['qual_type'] = df_all['qual_type'].fillna('none')

In [168]:
df_all['degree'].nunique()

10385

In [170]:
df_all.to_csv('data_v3.1.csv')

In [ ]:
df_all = df_all.drop('industry','hq_country')

In [6]:
industry['industry'].unique()

array(['human resources', 'banking',
       'information technology and services', 'marketing and advertising',
       'legal services', 'management consulting',
       'computer & network security', 'financial services',
       'online media', 'e-learning', 'food production',
       'computer software', 'plastics', 'logistics and supply chain',
       'capital markets', 'broadcast media', 'sports', 'law practice',
       'accounting', 'printing', 'consumer services',
       'security and investigations', 'cosmetics', 'sporting goods',
       'insurance', 'real estate', 'venture capital & private equity',
       'construction', 'investment banking', 'arts and crafts',
       'outsourcing/offshoring', 'mechanical or industrial engineering',
       'internet', 'chemicals', 'tobacco', 'pharmaceuticals',
       'environmental services', 'computer hardware', 'media production',
       'oil & energy', 'higher education', 'restaurants',
       'education management', 'retail', 'newspapers',
 